# Mandril with multiple experts, and suboptimal experts

In [1]:
from mab.expertTest import ExpertTest

In [2]:
paths = {
    "perfect": "banditk5n10/perfect",
    "rand_from_2_best": "banditk5n10/rand_from_2_best",
    "rand_from_3_best": "banditk5n10/rand_from_3_best",
}

In [3]:
perfect = ExpertTest(paths["perfect"])
rand_from_2_best = ExpertTest(paths["rand_from_2_best"])
rand_from_3_best = ExpertTest(paths["rand_from_3_best"])

In [4]:
def disaply_task(logs, ind):
    display(logs["tasks"][ind])
    display(logs["train_returns"][ind])
    display(logs["valid_returns"][ind])

In [5]:
display(perfect.logs["tasks"].shape)
display(perfect.logs["train_returns"].shape)
display(perfect.logs["valid_returns"].shape)


disaply_task(perfect.logs, 0)
disaply_task(perfect.logs, 1)
disaply_task(perfect.logs, 2)
disaply_task(perfect.logs, 101)

# display(np.mean(logs["train_returns"],1))
# display(np.mean(logs["valid_returns"],1))

(2000,)

(2000, 10)

(2000, 10)

{'mean': array([0.80739044, 0.51450007, 0.19111815, 0.18687478, 0.77252164])}

array([1., 0., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

...

array([1., 1., 1., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

array([2., 2., 2., 2., 2., 2., 2., 2., 1., 2.], dtype=float32)

{'mean': array([0.28384513, 0.04069452, 0.8575478 , 0.31959087, 0.57888191])}

array([1., 1., 1., 1., 0., 0., 1., 1., 1., 1.], dtype=float32)

array([1., 1., 2., 1., 2., 1., 2., 1., 2., 2.], dtype=float32)

In [6]:
disaply_task(rand_from_2_best.logs, 0)
disaply_task(rand_from_2_best.logs, 1)
disaply_task(rand_from_2_best.logs, 2)
disaply_task(rand_from_2_best.logs, 101)

# display(np.mean(logs["train_returns"],1))
# display(np.mean(logs["valid_returns"],1))

{'mean': array([0.80739044, 0.51450007, 0.19111815, 0.18687478, 0.77252164])}

array([0., 0., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

array([2., 2., 2., 2., 2., 1., 1., 2., 2., 2.], dtype=float32)

{'mean': array([0.05182789, 0.06952621, 0.51517512, 0.68854365, 0.45962809])}

array([1., 0., 0., 0., 0., 1., 0., 1., 0., 0.], dtype=float32)

...

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.], dtype=float32)

{'mean': array([0.28384513, 0.04069452, 0.8575478 , 0.31959087, 0.57888191])}

array([1., 1., 0., 1., 1., 1., 0., 1., 1., 0.], dtype=float32)

array([2., 1., 2., 1., 1., 2., 2., 1., 2., 2.], dtype=float32)